## packages and functions

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go  
from datetime import datetime
import colorsys

In [12]:
def generate_rainbow_colors(N, saturation=1.0, brightness=1.0, gap=.2):
    '''Generates N rainbow colors with specified saturation and brightness  
    which can be used in Plotly charts.

    Parameters
    ------------
    N : int or list-like
        number of colors or list-like item

    saturation : float
        
    brightness : float

    gap : float
        if <1, prevents the far right part of the spectrum  from coinciding with 
        the violet one in the left   

    Returns
    -------
    colors: list of strings 
        e.g. ['rgb(255,0,0)', 'rgb(51,255,0)', 'rgb(0,102,255)']

    Example
    -------
    # Generates and plots 20 colors:
    N = 20
    rainbow_colors = generate_rainbow_colors(N)
    fig = go.Figure(
        data=[go.Bar(x=list(range(N)), y=[1]*N, marker_color=rainbow_colors)])
    fig.show()   
    '''
    colors = []
    max_hue = 1.0-gap

    if isinstance(N,int): 
        NN = np.linspace(0,1,N) 
    else:  # if list-like ...
        # checks if list constists only on numerics
        if all(isinstance(x, (int, float)) for x in N):
            try:
                NN = (N - min(N))/(max(N)-min(N))
            except Exception as err_msg:
                NN = np.linspace(0,1,len(N))
                print(err_msg)
        else:
            NN = np.linspace(0,1,len(N))

    NN = max_hue*(1-NN)
    for n in NN:
        if isinstance(n,float) & (not np.isnan(n)):
            rgb = colorsys.hsv_to_rgb(n, saturation, brightness)
            colors.append(f'rgb({rgb[0]*255:.0f},{rgb[1]*255:.0f},{rgb[2]*255:.0f})')
        else:
            # replacing occasional nans with grey
            colors.append('grey')
    return colors

## reading/processing data

ref.: [Archive resource accounts - The Norwegian Offshore Directorate](https://www.sodir.no/en/facts/resource-accounts/arcive-resource-acconts/)  
rec. oil = mill. sm3 OE  
rec. gas = bill. sm3 gas  
rec. OE = mill. sm3   

In [13]:
df = pd.read_excel('_data.xlsx', sheet_name='data', usecols='A:H')
df['field'] = df['field'].str.upper()
df['field'] = df['field'].str.strip()
df['date'] = pd.to_datetime(df['date'])
df['Norwegian share'] = df['Norwegian share'].fillna(1.0)
# df = df[df.date>=datetime(1980,1,1)]
df = df.sort_values(by=['date','field'])
df = df.set_index('field')

df = df[~df.index.str.contains('*', regex=False)]
df = df.drop(index='TROLL BRENT B')

# optional correction 
df['rec. OE'] /= df['Norwegian share']

# sdf = selected df
sdf = df[df['date']==datetime(2024,12,31)].copy()
sdf['log_rr'] = np.log10(sdf['rec. OE'])
sdf['color'] = np.log10(sdf['rec. OE']).clip(-0.5,3) # 
sdf['color'] = generate_rainbow_colors(N=sdf['color'])
df = df.loc[sdf.index] # droping everything

df['ratio'] = np.nan
df['clr'] = None
sdf['dRm'] = np.nan
sdf['dR'] = np.nan
sdf['std(R)'] = np.nan
for f in sdf.index:
    Rmax = df.loc[f,'rec. OE'].max()
    df.loc[f,'ratio'] = round(df.loc[f,'rec. OE'] / Rmax, 3)
    df.loc[f,'clr'] = sdf.at[f,'color']
    df.loc[f,'log_rr'] = sdf.at[f,'log_rr']
    if isinstance(df.loc[f], pd.DataFrame):
        if len(df.loc[f,['date']])<2: continue
        R = df.loc[f,'ratio']
        dR = R.diff()
        sdf.loc[f,'dRm'] = round(dR.mean(),4)
        sdf.loc[f,'dR'] = R.iloc[-1] - R.iloc[0]
        R0 = df.loc[f,'rec. OE'].iloc[0]
        Rf = df.loc[f,'rec. OE'].iloc[-1]
        sdf.loc[f,'dR2'] = (Rf - Rmax) / Rf
        sdf.loc[f,'std(R)'] = R.std()

## four fields

In [14]:
theme = 'plotly_white'
markers=['circle','square','diamond','cross','triangle-up','triangle-down',
         'pentagon','star', 'hexagram', 'x', 'diamond-tall']*20

sdf = sdf.sort_values(by='rec. OE', ascending=False)
fig=go.Figure()
c = 0
# for n,f in enumerate(sdf.index):
for n,f in enumerate(['EKOFISK', 'OSEBERG', 'GRANE', 'MARIA']):
    if len(df.loc[f,['date']])<2: continue
    fig.add_trace(
        go.Scattergl(
            x=df.loc[f,'date'].dt.year, y=df.loc[f,'ratio'],
            mode='lines+markers', name=f,
            line=dict(color=sdf.at[f,'color']),
            marker={'symbol': markers[c]+ ('-open' if n%2==0 else ''), 
                    'opacity': .5, 'line': {'width': 1}, 'size':12}
        ))
    c += 1
fig.update_layout(
    template=theme, yaxis_tickformat=".0%", font_size = 16,
    margin={"r": 20, "t": 0, "l": 20, "b": 20}, 
    legend=dict(
        groupclick="toggleitem", x=0.02, y=.98, xanchor='left', yanchor='top'),    
    # showlegend=False,
    yaxis_title='reserves estimates normalized to max. value'
    )
#fig.write_html('./4 fields.html')
fig.show(renderer='browser')

## chart with all fields

In [15]:
df = df.sort_index().sort_values(by='date')
sdf = sdf.sort_index(ascending=False)
fig=go.Figure()
c = 0
# for n,f in enumerate(sdf.index):
for n,f in enumerate(reversed(sdf.index)):
    if len(df.loc[f,['date']])<2: continue
    fig.add_trace(
        go.Scattergl(
            x=df.loc[f,'date'].dt.year, y=df.loc[f,'ratio'],
            mode='lines+markers', name=f,
            line=dict(color=sdf.at[f,'color']),
            marker={'symbol': markers[c]+ ('-open' if n%2==0 else ''), 
                    'opacity': .5, 'line': {'width': 1}, 'size':12}
        ))
    c += 1
fig.update_layout(
    template=theme, yaxis_tickformat=".0%", font_size = 16,
    margin={"r": 20, "t": 0, "l": 20, "b": 20}, 
    # showlegend=False,
    yaxis_title='reserves estimates normalized to max. value'
    )
fig.show(renderer='browser')
#fig.write_html("./all fields.html")

## chart with all fields and colorbar

In [16]:
markers=['circle','square','diamond','cross','triangle-up','triangle-down',
         'pentagon','star', 'hexagram', 'x', 'diamond-tall']*20

sdf = sdf.sort_values(by='rec. OE', ascending=False)

# Get the range of log_rr values
min_log_rr = sdf['log_rr'].min()
max_log_rr = sdf['log_rr'].max()

# Create tick values in log space
# Major ticks at powers of 10 (with labels)
major_real_vals = [1, 10, 100, 1000]
major_log_vals = [np.log10(v) for v in major_real_vals if min_log_rr <= np.log10(v) <= max_log_rr]
major_labels = [str(int(10**v)) for v in major_log_vals]

# Add the maximum value as a major tick
max_real_val = 10**max_log_rr
major_log_vals.append(max_log_rr)
major_labels.append(f'{max_real_val:.0f}')  # Format as integer

# Minor ticks (without labels) - e.g., 2,3,4,5,6,7,8,9, 20,30,...
minor_real_vals = []
for power in range(int(np.floor(min_log_rr)), int(np.ceil(max_log_rr)) + 1):
    base = 10**power
    minor_real_vals.extend([base * i for i in range(2, 10)])  # 2,3,4,5,6,7,8,9 times base

minor_log_vals = [np.log10(v) for v in minor_real_vals if min_log_rr <= np.log10(v) <= max_log_rr]

# Combine for tickvals (all positions) and ticktext (labels only for major)
all_tick_vals = sorted(major_log_vals + minor_log_vals)
all_tick_text = []
for tick in all_tick_vals:
    if tick in major_log_vals:
        idx = major_log_vals.index(tick)
        all_tick_text.append(major_labels[idx])
    else:
        all_tick_text.append('')  # Empty string for minor ticks

fig=go.Figure()
c = 0
for n,f in enumerate(reversed(sdf.index)):
    if len(df.loc[f,['date']])<2: continue
    fig.add_trace(
        go.Scattergl(
            x=df.loc[f,'date'].dt.year, y=df.loc[f,'ratio'],
            mode='lines', name=f,
            line=dict(color=sdf.at[f,'color']),
            marker={'symbol': markers[c]+ ('-open' if n%2==0 else ''), 
                    'opacity': .5, 'line': {'width': 1}, 'size':12}
        ))
    c += 1

# Add a dummy trace for the colorbar using log_rr values
fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(
            colorscale='Rainbow',
            showscale=True,
            cmin=min_log_rr,
            cmax=max_log_rr,
            colorbar=dict(
                title="current<br>reserves<br>(mln. sm3 OE)",
                thickness=25, len=0.9, x=1.02,
                tickvals=all_tick_vals,
                ticktext=all_tick_text,
                ticks='outside',
                ticklen=5,
                tickfont=dict(size=15),  
                titlefont=dict(size=16),                 
            )
        ),
        showlegend=False,hoverinfo='none'
    )
)

fig.update_layout(
    template='plotly_white', yaxis_tickformat=".0%", font_size = 16,
    margin={"r": 90, "t": 0, "l": 20, "b": 20},
    showlegend=False,
    yaxis_title='reserves estimates normalized to max. value'
    )
fig.show(renderer='browser')
#fig.write_html("./all fields + colorbar.html")

## N largest vs. N smallest

In [17]:
nfields = 10

# largest
fig=go.Figure()
c = 0
for n,f in enumerate(sdf.head(nfields).index):
    if len(df.loc[f,['date']])<2: continue
    clr = sdf.at[f,'color']
    fig.add_trace(
        go.Scattergl(
            x=df.loc[f,'date'].dt.year, y=df.loc[f,'ratio'],
            mode='markers+lines', name=f, line=dict(color=clr),
            legendgroup=f'{nfields} largest fields',
            legendgrouptitle_text=f'{nfields} largest fields',   
            marker={'symbol': markers[c], 
                    'opacity': 0.7, 'line': {'width': 1.5}, 'size': 12}
        ))
    c += 1
# smallest
c = 0
for n,f in enumerate(sdf.tail(nfields).index):
    if len(df.loc[f,['date']])<2: continue
    # if f not in ind: continue
    clr = sdf.at[f,'color']
    fig.add_trace(
        go.Scattergl(
            x=df.loc[f,'date'].dt.year, y=df.loc[f,'ratio'],
            mode='markers+lines', name=f, line=dict(color=clr, dash='dot'),
            legendgroup=f'{nfields} smallest fields',
            legendgrouptitle_text=f'{nfields} smallest fields',
            marker={'symbol': markers[c]+ '-open', 
                    'line': {'width': 1.5}, 'size': 12}
        ))
    c += 1

fig.update_layout(
    template=theme, yaxis_tickformat=".0%", font_size = 16,
    margin={"r": 20, "t": 0, "l": 20, "b": 20}, 
    legend_groupclick='toggleitem',
    yaxis_title='reserves estimates normalized to max. value',
    )
fig.show(renderer='browser')
#fig.write_html(f'./{nfields} smalles vs. {nfields} largest.html')

## reserves changes vs. field size

In [18]:
fig = px.scatter(sdf.reset_index(), x='rec. OE', y='dR', log_x=True, 
                 hover_data = 'field', color_continuous_scale='rainbow', 
                 symbol_sequence=['circle-open'])
fig.update_layout(
    template='plotly_white', 
    title='reserves changes ([initial] - [final]  as % of max value) vs. field size',
    xaxis_title='most recent reserve estimate (mill. sm3 OE)',
    yaxis_title='(Rf - R0)/Rmax', yaxis_tickformat=".0%",
    # margin={"r": 20, "t": 0, "l": 20, "b": 20}
    font_size = 16,
)
fig.show(renderer='browser')

In [19]:
sdf[['log_rr','dR']].corr()

,log_rr,dR
log_rr,1.000000,0.633153
dR,0.633153,1.000000
